In [36]:
from sklearn.datasets import load_files
from collections import Counter
from nltk.tokenize import word_tokenize
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
pd.options.mode.chained_assignment = None

In [37]:
class AutoCorrect:
    def from_file(self,file):
        vocablist=[]
        with open(file,encoding='utf-8-sig',mode='r') as file:
            lines=file.readlines()
            for line in lines:
                vocablist +=[word for word in re.findall(r'\w+',line) if len(word)>1]
        vocab_counter=Counter(vocablist)
        totalwords=sum(vocab_counter.values())
        self.vocab=set(vocablist)
        self.word_prob={word:vocab_counter[word]/totalwords for word in vocab_counter.keys()}

    def from_dir(self,dir):
        dataset=load_files(dir,encoding='utf-8-sig')
        vocablist=[]
        for doc in dataset['data']:
            vocablist +=[word for word in re.findall(r'\w+',doc) if len(word)>1]
        vocab_counter=Counter(vocablist)
        totalwords=sum(vocab_counter.values())
        self.vocab=set(vocablist)
        self.word_prob={word:vocab_counter[word]/totalwords for word in vocab_counter.keys()}


    def _level_one_edits(self,word):
        letters=u'ابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإئ'
        splits=[(word[:part],word[part:]) for part in range(0,len(word))]
        deletes=[part1+part2[1:] for part1,part2 in splits if part2]
        swaps=[part1+part2[0]+part2[1]+part2[2:] for part1,part2 in splits if len(part2) > 1]
        replaces=[part1+c+part2[1:] for part1,part2 in splits if part2 for c in letters]
        inserts=[part1+c+part2 for part1,part2 in splits for c in letters]
        return set(deletes+swaps+replaces+inserts)

    def _level_two_edits(self,word):
        return set(e2 for e1 in self._level_one_edits(word) for e2 in self._level_one_edits(e1))

    def check(self,word):
        if word in self.vocab:
            return True
        candidates =list(self._level_one_edits(word))+list(self._level_two_edits(word))+[word]
        valid_candidates = [w for w in candidates if w in self.vocab]
        if len(valid_candidates) > 0 :
            return sorted([(c, self.word_prob[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)
        else:
            return False

    def correct(self,word):
        if word in self.vocab:
            return word
        candidates =list(self._level_one_edits(word))+list(self._level_two_edits(word))+[word]
        valid_candidates = [w for w in candidates if w in self.vocab]
        if len(valid_candidates) > 0 :
            return sorted([(c, self.word_prob[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)[0][0]
        else:
            return word


In [38]:
autoCorrect=AutoCorrect()

In [39]:
autoCorrect.from_dir('Khaleej-2004')

In [40]:
df=pd.read_csv('Cleaned_data.csv')
df.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,52.0,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,56.0,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,54.0,NaN
4,الكاتب محمود سالم,لغز المنزل رقم 98,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0,NaN


In [41]:
df[~(df["Quotation"].isna())]

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
7,الكاتب هارون يحيى,التضحية عند الحيوان,كتب الأدب العربي-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,5.00,160.0,ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات...
8,الكاتب هارون يحيى,معجزة الذرة,كتب تطوير الذات-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3.41,136.0,إن هذا الكتاب يبين وبالدليل العلمي أن الذي خلق...
9,الكاتب هارون يحيى,لا تتجاهل,كتب الأدب العربي,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3.00,100.0,هذه الحقيقة بالإضافة إلى حقائق كونية أخرى يستع...
10,الكاتب هارون يحيى,الحياة في سبيل الله,كتب اسلامية,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,2.11,100.0,أما المنافقون فهم في سبيل الفوز بما قال عنه ال...
11,الكاتب هارون يحيى,العظمة في كل مكان,كتب الأدب العربي,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,5.30,140.0,ولو سال عن كيفية التي يتحرك بها كل شيء هذا الك...
...,...,...,...,...,...,...,...,...
1182,الكاتب الياس ابو شبكة,القيثارة لإلياس أبو شبكة,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,5.00,198.0,لقد فتك ت بك يد أثيمة في بلاد الغربة بعيد ا عن...
1183,الكاتب الياس ابو شبكة,المريض الصامت,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,1.00,30.0,إن ما تقرأ في هذه الصفحات لهو حقيقة أليمة نطقت...
1185,الكاتب الياس ابو شبكة,العمال الصالحون,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,1.00,80.0,كان لبيب راغب ولد ا في العاشرة من سنيه جميل ال...
1186,الكاتب الياس ابو شبكة,الألحان ومقتطفات من غلواء,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,13.00,22.0,إلى الحصاد جنى الجهاد قلب البلاد يحيا بنا هيا ...


In [42]:
#Auto Correct data
book_names=[]
book_Quotation=[]
for record in df["Book_Name"]:
    record=re.sub("\d","",record)
    words=word_tokenize(record)
    words_new=[]
    for word in words:
        if len(word) < 2:
            words_new.append(autoCorrect.correct(word))
        else:
            words_new.append(word)
    book_names.append(' '.join(words_new))

for record in df[~(df["Quotation"].isna())]["Quotation"]:
    record=re.sub("\d","",record)
    words=word_tokenize(record)
    words_new=[]
    for word in words:
        if len(word) < 2:
            words_new.append(autoCorrect.correct(word))
        else:
            words_new.append(word)
    book_Quotation.append(' '.join(words_new))

In [43]:
df["Book_Name"]=book_names
df[~(df["Quotation"].isna())]["Quotation"]=book_Quotation

In [44]:
#split data to two models one with Quotation and with not
df1=df[df["Quotation"].isna()]
df2=df[~(df["Quotation"].isna())]

In [45]:
df1=df1[df.columns[:-1]]
df1.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,52.0
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,56.0
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,54.0
4,الكاتب محمود سالم,لغز المنزل رقم,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0


In [46]:
df1['Book_Category'].value_counts()

كتب اسلامية                                                                                 1202
روايات بوليسية                                                                               618
كتب روايات عربية                                                                             578
كتب الأطفال                                                                                  228
العقيدة                                                                                      195
                                                                                            ... 
كتب اسلامية-كتب الأدب العربي-كتب التاريخ-كتب التاريخ الإسلامي                                  1
كتب القصص القصيرة المترجمة-كتب الروايات العالمية المترجمة-كتب روايات عربية                     1
كتب الروايات العالمية المترجمة-كتب الأدب العالمي المترجم-كتب المسرحيات العالمية المترجمة       1
الثقافة العامة-كتب السياسة                                                                     1
كتب التنمية الذاتية-كتب تطوير 

In [47]:
df2["Book_Category"].value_counts()

كتب الأدب العربي                                                     38
كتب الأدب العربي-كتب الروايات العالمية المترجمة-كتب روايات عربية     31
كتب الأدب العربي-كتب روايات عربية                                    23
كتب اسلامية                                                          23
كتب الروايات العالمية المترجمة-كتب روايات عربية                      16
كتب التاريخ-كتب التاريخ القديم                                       15
كتب الفلسفة والمنطق                                                  10
كتب اسلامية-كتب الأدب العربي                                          8
كتب الأدب العربي-كتب التاريخ                                          2
كتب الروايات العالمية المترجمة                                        1
كتب الأدب العربي-كتب المسرحيات العالمية المترجمة-كتب روايات عربية     1
كتب الفلسفة والمنطق-كتب علم النفس                                     1
روايات بوليسية-كتب الروايات العالمية المترجمة                         1
كتب أساطير-كتب التاريخ-كتب التاريخ القديم                       

In [48]:
#prepare the training for model 1
x1_test=df1[df1["Book_Category"].isna()]["Book_Name"].tolist()
x1_train=df1[~(df1["Book_Category"].isna())]["Book_Name"].tolist()
y1_train=df1[~(df1["Book_Category"].isna())]["Book_Category"].tolist()

In [49]:
pipe=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",LogisticRegression(max_iter=2000))])

In [50]:
pipe.fit(x1_train,y1_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', LogisticRegression(max_iter=2000))])

In [51]:
pipe.score(x1_train,y1_train)

0.5893324036953111

In [52]:
pipe.predict(["لغز القبر الملكي"])

array(['روايات بوليسية'], dtype='<U105')

In [53]:
pipe2=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",MultinomialNB())])

In [54]:
pipe2.fit(x1_train,y1_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', MultinomialNB())])

In [55]:
pipe2.score(x1_train,y1_train)

0.5138574167683458

In [56]:
pipe2.predict(["لغز القبر الملكي"])

array(['روايات بوليسية'], dtype='<U105')

In [57]:
pipe3=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",RandomForestClassifier(n_estimators=700))])
pipe3.fit(x1_train,y1_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', RandomForestClassifier(n_estimators=700))])

In [58]:
pipe3.score(x1_train,y1_train)

0.9855325082795886

In [59]:
pipe3.predict(["لغز القبر الملكي"])

array(['روايات بوليسية'], dtype='<U105')

In [60]:
#prepare the training for model 2
x2_test=df2[df2["Book_Category"].isna()]["Quotation"].tolist()
x2_train=df2[~(df2["Book_Category"].isna())]["Quotation"].tolist()
y2_train=df2[~(df2["Book_Category"].isna())]["Book_Category"].tolist()

In [61]:
pipe=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",LogisticRegression(max_iter=2000))])
pipe.fit(x2_train,y2_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', LogisticRegression(max_iter=2000))])

In [62]:
pipe.score(x2_train,y2_train)

0.7724867724867724

In [63]:
pipe.predict(["ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات يثير الاستغراب والدهشة ويبعث على التأمل ذلك أنه سلوك ينطق بالحكمة والعقل ويبين عن وعي خارق وبعد نظر عميق وعند قراءة هذه النماذج سوف يقفز إلى أذهانكم ..."])

array(['كتب الأدب العربي'], dtype='<U65')

In [64]:
pipe2 = Pipeline([("Vector", TfidfVectorizer(encoding='utf-8-sig', lowercase=False)), ("Model", MultinomialNB())])

In [65]:
pipe2.fit(x2_train,y2_train)
pipe2.score(x2_train,y2_train)

0.6825396825396826

In [66]:
pipe2.predict(["ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات يثير الاستغراب والدهشة ويبعث على التأمل ذلك أنه سلوك ينطق بالحكمة والعقل ويبين عن وعي خارق وبعد نظر عميق وعند قراءة هذه النماذج سوف يقفز إلى أذهانكم ..."])

array(['كتب الأدب العربي'], dtype='<U65')

In [67]:
pipe3=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",RandomForestClassifier(n_estimators=700))])
pipe3.fit(x2_train,y2_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', RandomForestClassifier(n_estimators=700))])

In [68]:
pipe3.score(x2_train,y2_train)

0.9682539682539683

In [69]:
pipe3.predict(["ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات يثير الاستغراب والدهشة ويبعث على التأمل ذلك أنه سلوك ينطق بالحكمة والعقل ويبين عن وعي خارق وبعد نظر عميق وعند قراءة هذه النماذج سوف يقفز إلى أذهانكم ..."])

array(['كتب الأدب العربي'], dtype='<U65')